In [21]:
# -*- coding: utf-8 -*-

from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread,Lock,Semaphore
import requests

sem = Semaphore(10)  # 此变量用于控制最大并发数
dllock = Lock()  #此变量用来控制实时行情推送中落数据到本地的锁

In [22]:
pd.options.display.width = 320
pd.options.display.max_columns = None

In [23]:
# 登录函数
def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("blsj003","178583")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')
        
thslogindemo()

-201
登录失败


# 数据池THS_DP

In [24]:
today_str = datetime.today().strftime('%Y-%m-%d')
lastday_str = THS_Date_Offset('212001','dateType:0,period:D,offset:-1,dateFormat:0,output:singledate',today_str).data
startday_str = THS_Date_Offset('212001','dateType:0,period:D,offset:-60,dateFormat:0,output:singledate',today_str).data
today_str,lastday_str,startday_str

('2023-02-09', '2023-02-08', '2022-11-09')

In [25]:
print('today:{}'.format(today_str))
data_dp = THS_DP('block','{};001005345'.format(today_str),'date:Y,thscode:Y,security_name:Y').data
data_dp

today:2023-02-09


,DATE,THSCODE,SECURITY_NAME
0,2023/02/09,689009.SH,九号公司
1,2023/02/09,688981.SH,中芯国际
2,2023/02/09,688819.SH,天能股份
3,2023/02/09,688800.SH,瑞可达
4,2023/02/09,688799.SH,华纳药厂
...,...,...,...
4911,2023/02/09,000006.SZ,深振业A
4912,2023/02/09,000005.SZ,ST星源
4913,2023/02/09,000004.SZ,ST国华
4914,2023/02/09,000002.SZ,万科A


# 实时行情： THS_RQ

In [26]:
thscode = data_dp['THSCODE'].tolist()
jsonIndicator = ['open', 'latest']
data_rq = THS_RQ(thscode=thscode, jsonIndicator=";".join(jsonIndicator)).data
flag_ss = data_rq['thscode'].str.startswith("0") | data_rq['thscode'].str.startswith("6")
data_rq = data_rq[flag_ss]
data_rq = data_rq[data_rq['latest']-data_rq['open'] > 0]
data_rq

,time,thscode,open,latest
1,2023-02-09 15:59:54,688981.SH,42.50,43.60
2,2023-02-09 15:59:42,688819.SH,38.48,39.01
3,2023-02-09 15:59:50,688800.SH,113.42,116.31
5,2023-02-09 15:59:49,688798.SH,111.11,119.56
6,2023-02-09 15:59:41,688793.SH,48.63,51.07
...,...,...,...,...
4911,2023-02-09 16:30:00,000006.SZ,5.43,5.59
4912,2023-02-09 16:30:57,000005.SZ,1.81,1.82
4913,2023-02-09 16:30:51,000004.SZ,10.21,10.22
4914,2023-02-09 16:30:54,000002.SZ,17.49,17.67


# 日期序列： THS_DS


In [27]:
# 昨天开盘价，收盘价
thscode = data_rq['thscode'].tolist()
data_last = THS_DS(thscode,'ths_open_price_stock;ths_close_price_stock','100;100','',lastday_str, lastday_str).data
data_last

,time,thscode,ths_open_price_stock,ths_close_price_stock
0,2023-02-08,688981.SH,43.20,42.62
1,2023-02-08,688819.SH,38.22,38.42
2,2023-02-08,688800.SH,112.80,114.11
3,2023-02-08,688798.SH,117.35,114.99
4,2023-02-08,688793.SH,49.93,48.92
...,...,...,...,...
3112,2023-02-08,000006.SZ,5.39,5.47
3113,2023-02-08,000005.SZ,1.83,1.82
3114,2023-02-08,000004.SZ,10.43,10.30
3115,2023-02-08,000002.SZ,17.54,17.50


In [28]:
data_res_a = pd.merge(data_rq[['thscode','open','latest','time']], 
                      data_last[['thscode','ths_open_price_stock','ths_close_price_stock']], 
                      on=['thscode'], 
                      how="left")
data_res_a.columns = ['thscode','open','latest','datetime','last_day_open','last_day_close']
data_res_a["老A"] = (data_res_a["latest"] - data_res_a["last_day_close"]) / (data_res_a["last_day_open"] - data_res_a["last_day_close"])
data_res_a["老A"] = data_res_a["老A"].round(2)
data_res_a = data_res_a[data_res_a['老A'].between(0.5, 1.5)] 
data_res_a

,thscode,open,latest,datetime,last_day_open,last_day_close,老A
9,688779.SH,16.20,16.37,2023-02-09 15:59:33,16.33,16.17,1.25
10,688778.SH,84.76,85.94,2023-02-09 15:59:49,85.68,84.23,1.18
12,688772.SH,22.86,22.88,2023-02-09 15:59:52,22.81,22.95,0.50
14,688767.SH,48.26,48.48,2023-02-09 15:59:42,48.63,48.26,0.59
22,688701.SH,8.85,8.88,2023-02-09 15:59:59,8.88,8.83,1.00
...,...,...,...,...,...,...,...
3096,000034.SZ,23.40,24.45,2023-02-09 16:30:27,24.99,23.55,0.62
3098,000031.SZ,3.73,3.76,2023-02-09 16:30:06,3.76,3.74,1.00
3102,000026.SZ,11.88,12.05,2023-02-09 16:30:57,12.01,11.90,1.36
3104,000020.SZ,10.21,10.29,2023-02-09 16:30:15,10.34,10.20,0.64


# 60日均价

In [29]:
stock_list = data_res_a['thscode'].tolist()
stock_per = [stock_list[i:i + int(len(stock_list) / 10)] for i in range(0, len(stock_list), int(len(stock_list) / 10))]

In [30]:
data_ds = pd.DataFrame()
for eachlist in stock_per:
    data_tmp = THS_DS(','.join(eachlist), 'ths_close_price_stock', '100', '', startday_str, lastday_str).data
    data_ds = data_ds.append(data_tmp)

In [31]:
data_ds

,time,thscode,ths_close_price_stock
0,2022-11-09,688779.SH,16.02
1,2022-11-10,688779.SH,16.06
2,2022-11-11,688779.SH,16.08
3,2022-11-14,688779.SH,15.82
4,2022-11-15,688779.SH,16.33
...,...,...,...
295,2023-02-02,000019.SZ,7.89
296,2023-02-03,000019.SZ,7.83
297,2023-02-06,000019.SZ,7.79
298,2023-02-07,000019.SZ,7.82


In [32]:
# 60日均价
# data_ds = THS_DS(thscode, 'ths_close_price_stock', '100', '', start_day, lastday_str).data

data_avg_60 = data_ds[['thscode', 'ths_close_price_stock']].groupby(['thscode'], as_index=False)['ths_close_price_stock'].mean().round(2)
data_avg_60.columns = ['thscode', 'ths_m60_price_stock']
data_avg_60

,thscode,ths_m60_price_stock
0,000019.SZ,7.61
1,000020.SZ,9.59
2,000026.SZ,10.82
3,000031.SZ,3.79
4,000034.SZ,23.31
...,...,...
980,688701.SH,9.08
981,688767.SH,51.25
982,688772.SH,21.40
983,688778.SH,84.56


In [33]:
data_res_b = pd.merge(data_res_a, data_avg_60, on=['thscode'], how="left")
# data_res_b = data_res_b[data_res_b['latest']-data_res_b['ths_m60_price_stock'] > 0]
data_res_b["老C"] = data_res_b['latest'] / data_res_b['ths_m60_price_stock']
data_res_b["老C"] = data_res_b["老C"].round(2)
data_res_b =  data_res_b[data_res_b['老C'].between(0.7, 1.2)]
data_res_b

,thscode,open,latest,datetime,last_day_open,last_day_close,老A,ths_m60_price_stock,老C
0,688779.SH,16.20,16.37,2023-02-09 15:59:33,16.33,16.17,1.25,15.92,1.03
1,688778.SH,84.76,85.94,2023-02-09 15:59:49,85.68,84.23,1.18,84.56,1.02
2,688772.SH,22.86,22.88,2023-02-09 15:59:52,22.81,22.95,0.50,21.40,1.07
3,688767.SH,48.26,48.48,2023-02-09 15:59:42,48.63,48.26,0.59,51.25,0.95
4,688701.SH,8.85,8.88,2023-02-09 15:59:59,8.88,8.83,1.00,9.08,0.98
...,...,...,...,...,...,...,...,...,...
980,000034.SZ,23.40,24.45,2023-02-09 16:30:27,24.99,23.55,0.62,23.31,1.05
981,000031.SZ,3.73,3.76,2023-02-09 16:30:06,3.76,3.74,1.00,3.79,0.99
982,000026.SZ,11.88,12.05,2023-02-09 16:30:57,12.01,11.90,1.36,10.82,1.11
983,000020.SZ,10.21,10.29,2023-02-09 16:30:15,10.34,10.20,0.64,9.59,1.07


# 基础数据：THS_BD

In [34]:
thscode = data_res_b['thscode'].tolist()
data_bd = THS_BD(thscode,'ths_stock_short_name_stock;ths_listedsector_stock;ths_the_csrc_industry_stock',';;100,{}'.format(today_str)).data
data_bd

,thscode,ths_stock_short_name_stock,ths_listedsector_stock,ths_the_csrc_industry_stock
0,688779.SH,长远锂科,科创板,
1,688778.SH,厦钨新能,科创板,
2,688772.SH,珠海冠宇,科创板,
3,688767.SH,博拓生物,科创板,
4,688701.SH,卓锦股份,科创板,社会服务业
...,...,...,...,...
958,000034.SZ,神州数码,主板,综合类
959,000031.SZ,大悦城,主板,房地产业
960,000026.SZ,飞亚达,主板,批发和零售贸易
961,000020.SZ,深华发A,主板,制造业


In [35]:
data_res_df = pd.merge(data_res_b, data_bd, on=['thscode'], how='left')
data_res_df = data_res_df[[
    'thscode', 'ths_stock_short_name_stock', 'ths_listedsector_stock',
    'ths_the_csrc_industry_stock', 'datetime', 'open', 'latest', 'last_day_open',
    'last_day_close', 'ths_m60_price_stock', '老A', '老C'
]]
data_res_df.columns = ['代码','名称','版块','行业','时间','今开','最新价','上开','上收','60均','老A','老C']
data_res_df = data_res_df.sort_values(['代码'])
data_res_df

,代码,名称,版块,行业,时间,今开,最新价,上开,上收,60均,老A,老C
962,000019.SZ,深粮控股,主板,制造业,2023-02-09 16:30:12,7.73,7.77,7.80,7.69,7.61,0.73,1.02
961,000020.SZ,深华发A,主板,制造业,2023-02-09 16:30:15,10.21,10.29,10.34,10.20,9.59,0.64,1.07
960,000026.SZ,飞亚达,主板,批发和零售贸易,2023-02-09 16:30:57,11.88,12.05,12.01,11.90,10.82,1.36,1.11
959,000031.SZ,大悦城,主板,房地产业,2023-02-09 16:30:06,3.73,3.76,3.76,3.74,3.79,1.00,0.99
958,000034.SZ,神州数码,主板,综合类,2023-02-09 16:30:27,23.40,24.45,24.99,23.55,23.31,0.62,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...
4,688701.SH,卓锦股份,科创板,社会服务业,2023-02-09 15:59:59,8.85,8.88,8.88,8.83,9.08,1.00,0.98
3,688767.SH,博拓生物,科创板,,2023-02-09 15:59:42,48.26,48.48,48.63,48.26,51.25,0.59,0.95
2,688772.SH,珠海冠宇,科创板,,2023-02-09 15:59:52,22.86,22.88,22.81,22.95,21.40,0.50,1.07
1,688778.SH,厦钨新能,科创板,,2023-02-09 15:59:49,84.76,85.94,85.68,84.23,84.56,1.18,1.02


In [36]:
data_res_df.to_excel("data_res_df_" + datetime.today().strftime('%Y%m%d') + ".xlsx", index=False)

In [37]:
THS_iFinDLogout()

0